In [2]:
import zipfile

In [3]:
zipfilename="./fashion_mnist_data.zip"
print(zipfile.is_zipfile(zipfilename))

True


In [ ]:
outpath="./"
with zipfile.ZipFile(zipfilename, 'r') as zfile:
    nl = zfile.namelist()
    il = zfile.infolist()
    for id,l in enumerate(nl):
        zfile.extract(l, path=outpath)
        print(l)

In [ ]:
import pickle 
f = open('./fashion_mnist_data/fashion_mnist.pickle', 'rb')
train, test = pickle.load(f)
f.close()

In [ ]:
train[0].shape

In [ ]:
temp=test[0][0].reshape([28,28])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(temp,'gray')
plt.show

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers,layers,losses
from tensorflow.keras.layers import Input,Dense
import tensorflow as tf
import numpy as np
%matplotlib inline

In [ ]:
noise_factor = 20

train_noise=np.zeros_like(train[0])
test_noise=np.zeros_like(test[0])


train_noise=train[0]+noise_factor*tf.random.normal(shape=train[0].shape)
test_noise=test[0]+noise_factor* tf.random.normal(shape=test[0].shape)

train_noise=tf.clip_by_value(train_noise,clip_value_min=0.,clip_value_max=255.)
test_noise=tf.clip_by_value(test_noise,clip_value_min=0.,clip_value_max=255.)

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("test")
    plt.imshow(tf.squeeze(test[0][i]))
    plt.gray()
plt.show()

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("noise")
    plt.imshow(tf.squeeze(test_noise[i]))
    plt.gray()
plt.show()

In [ ]:
train=list(train)
train[0]=train[0].astype('float32')/255.
train_noise=train_noise/255.
test=list(test)
test[0]=test[0].astype('float32')/255.
test_noise=test_noise/255.

In [ ]:
train=list(train)
test=list(test)
train[0]=np.array(train[0]).reshape(60000,28,28,1)
test[0]=np.array(test[0]).reshape(10000,28,28,1)
print(train[0].shape,test[0].shape)

train_noise=np.array(train_noise).reshape(60000,28,28,1)
test_noise=np.array(test_noise).reshape(10000,28,28,1)
print(train_noise.shape,test_noise.shape)

In [ ]:
class Denoise(Model):
    def __init__(self):
        super(Denoise, self).__init__()
        self.encoder = tf.keras.Sequential([layers.Input(shape=(28, 28, 1)),
                                            layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
                                            layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

        self.decoder = tf.keras.Sequential([layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
                                            layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
                                            layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Denoise()

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
train_=autoencoder.fit(train_noise, train[0],
                epochs=10,
                shuffle=True,
                validation_data=(test_noise, test[0]))

In [ ]:
encoded_imgs = autoencoder.encoder(test_noise).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
n = 12
plt.figure(figsize=(20, 4))
for i in range(n):

    
    ax = plt.subplot(2, n, i + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(test_noise[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(tf.squeeze(decoded_imgs[i]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()

In [ ]:
plt.plot(train_.history["loss"], label="Train Loss")
plt.plot(train_.history["val_loss"], label="Val Loss")
plt.legend()